In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import dgl
from dgl.utils import expand_as_pair
import dgl.function as fn
from dgl.utils import check_eq_shape

Любая НС модель состоит из модулей. Модули в `DGL` могут быть реализованы с использование фреймворков `Pytorch`, `MXNet` или `TensorFlow`. Описание модуля и работа с ним остаются такими, как предполагает соотвествующий фреймворк. Ключевое различие состоитв использование операций рассылки сообщений, которые реализованы в `DGL`.

Построение любого модуля в `Pytorch` состоит из 2 частей: описание метода `__init__` и описание метода `forward`.

В методе `__init__` необходимо:
1. Определить гиперпараметры модуля. Сюда относятся, среди прочего, размерности данных: размерность входа, скрытых слоев и выхода. Кроме этого существуют специфичные для GNN вещи, например, способ агрегации сообщений от соседей (`mean`, `sum` и т.д.) 
2. Зарегистрировать подмодули и настраиваемые параметры. Набор модулей в сети может меняться в зависимости от гиперпараметров.
3. Сбросить состояние (при необходимости): например, инициализировать веса обучаемых модулей.

Метод `forward` выполняет рассылку сообщений и расчеты. В отличие от стандартных моделей из `Pytorch`, в `DGL` `forward` принимает на вход еще и граф.

В методе `forward` необходимо:
1. Провести провести (граф, типы и т.д.). Типичный пример: проверка, что в графе нет узлов с 0 in-degree. В противном случае сообщения к ним не поступают и функция свертки будет возвращать нули.
2. Выполнить рассылку сообщений. В идеале модуль должен уметь работать с разными типами входных графов: homo- и heterogenious, subgraph blocks и т.д.
3. Обновить фичи (узлов или ребер)

В качестве примера рассматриваем SAGE:

![](./assets/img/13_dgl_building_modules_sage.png)

In [41]:
class SAGEConv(nn.Module):
    def __init__(self, n_inputs: int, n_outputs: int, aggregator_type: str, 
                 bias: bool = True, norm: callable = None, activation: callable = None):
        super().__init__()
        
        self.n_inputs = n_inputs
        self._in_src_feats, self._in_dst_feats = expand_as_pair(n_inputs)
        self.n_outputs = n_outputs
        self._aggre_type = aggregator_type
        self.norm = norm
        self.bias = bias
        self.activation = activation
        self._set_modules()
        self._reset_parameters()
    
    def _set_modules(self):
        if self._aggre_type != 'mean':
            raise KeyError('Aggregator type {} not supported.'.format(aggregator_type))

        self.fc_self = nn.Linear(self._in_dst_feats, self.n_outputs, bias=self.bias)
        self.fc_neigh = nn.Linear(self._in_src_feats, self.n_outputs, bias=self.bias)

    def _reset_parameters(self):
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_uniform_(self.fc_self.weight, gain=gain)
        nn.init.xavier_uniform_(self.fc_neigh.weight, gain=gain)

    def forward(self, G, features):
        # если граф однородный, то src_nodes = dst_nodes = all_nodes
        # если граф неоднородный, то его можно разбить на несколько двудольных графов
        # если обучение проводится на минибатчах, то работа будет вестись с подграфом типа block
        # expand_as_pair разибвает фичи на 2 тензора в зависимости от типа графа
        # после этого с ними можно работать, не обращая внимания на исходный тип графа
        feat_src, feat_dst = expand_as_pair(features, G)
        with G.local_scope():
            # aggregation
            # для однородных графов G.srcdata = G.dstdata = G.ndata
            G.srcdata['h'] = feat_src
            G.update_all(fn.copy_u('h', 'm'), fn.mean('m', 'h_n'))
            h_n = G.dstdata['h_n']

            # разбивка оригинальной формулы на 2 слагаемых
            out = self.fc_self(feat_dst) + self.fc_neigh(h_n)

            # активация + нормализация
            if self.activation is not None:
                out = self.activation(out)

            if self.norm is not None:
                out = self.norm(out)

            return out

In [42]:
from utils import create_edge_pred_graph

G = create_edge_pred_graph(n_nodes=100, n_edges=1000,
                           n_node_features=5, n_edge_features=2)

In [44]:
conv = SAGEConv(n_inputs=5, n_outputs=7, aggregator_type='mean', 
                activation=F.relu)

out = conv(G, G.ndata['feature'])
out.shape

torch.Size([100, 7])

In [56]:
# созданный модуль корректно обрабатывает работу с блоками
sampler = dgl.dataloading.MultiLayerFullNeighborSampler(n_layers=1)
train_ids = torch.arange(G.num_nodes())
dataloader = dgl.dataloading.NodeDataLoader(G, train_ids, sampler,
                                            batch_size=32, shuffle=True,
                                            drop_last=False)

input_nodes, output_nodes, blocks = next(iter(dataloader))  
block = blocks[0] 
block_f = block.srcdata['feature']
out = conv(block, block_f)
out.shape

torch.Size([32, 7])

torch.Size([32, 7])